# About the notebook
This script contains the machine analysis from the Griffin model published on GitHub (https://github.com/adoebley/Griffin_analyses). Here, their model slightly adjusted to use with my data. Similar results were gained when using binary classification with a balanced model.

In [1]:
import argparse
import sys
import os
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np

from sklearn.metrics import roc_curve,auc

# Functions

In [2]:
def import_data(in_file, meta_file):
    import pandas as pd
    import numpy as np
    from sklearn.preprocessing import StandardScaler
    
    data = pd.read_csv(in_file, sep='\t', index_col=0)
    meta = pd.read_csv(meta_file, sep='\t', index_col='sample_name')
    #data = data.set_index('sample')

    #get features and exclude all other columns
    features = data.columns[(data.columns.str.startswith('central_cov')) | (data.columns.str.startswith('mean_cov')) | (data.columns.str.startswith('amplitude')) | (data.columns.str.startswith('Ulz'))]
    print('Features',len(features))
    
    data = data.reset_index(drop=False)
    data[['sample','p','score']] = data['index'].str.split('_',2, expand=True)
    data = data.set_index('sample')
    data = pd.concat([data, meta], axis=1)
    
    data = data.sort_index()

    print('Total samples:',len(data))

    #scale data
    scaler = StandardScaler()
    scaler.fit(data[features])
    data[features] = scaler.transform(data[features])
    data[features].mean()
    
    #add tumor fraction groups
    data['tfx_group'] = 'none'
    a = 0
    for b in tfx_groups:
        tfx_group_name = str(a)+'-'+str(b)+'TFx'
        data['tfx_group'] = np.where((data['phenotype']==1) & (data['fraction']>=a) & (data['fraction']<b),tfx_group_name,data['tfx_group'])
        a=b
    #if group maxes don't go all the way to 1, add a group > max val
    if b<1:
        tfx_group_name = '>'+str(b)+'TFx'
        data['tfx_group'] = np.where((data['phenotype']==1) & (data['phenotype']==1) & (data['fraction']>=b),tfx_group_name,data['tfx_group'])
    #specify tfx group for healthy donors
    data['tfx_group'] = np.where((data['phenotype']==0),'Healthy',data['tfx_group'])
    
    return(data,features)


In [3]:
def import_data_adjusted(in_file_c, meta_file_c, in_file_h, meta_file_h):
    import pandas as pd
    import numpy as np
    from sklearn.preprocessing import StandardScaler
    
    data_c = pd.read_csv(in_file_c, sep='\t', index_col=0)
    meta_c = pd.read_csv(meta_file_c, sep='\t', index_col='sample_name')

    #get features and exclude all other columns
    features = data_c.columns[(data_c.columns.str.startswith('central_cov')) | (data_c.columns.str.startswith('mean_cov')) | (data_c.columns.str.startswith('amplitude')) | (data_c.columns.str.startswith('Ulz'))]
    print('Features',len(features))

    data_c = data_c.reset_index(drop=False)
    data_c[['sample','p','score']] = data_c['index'].str.split('_',2, expand=True)
    data_c = data_c.set_index('sample')
    data_c = pd.concat([data_c, meta_c], axis=1)
    
    data_h = pd.read_csv(in_file_h, sep='\t', index_col=0)
    meta_h = pd.read_csv(meta_file_h, sep='\t', index_col='sample_name')
    
    data_h = data_h.reset_index(drop=False)
    data_h[['sample','p','score']] = data_h['index'].str.split('_',2, expand=True)
    data_h = data_h.set_index('sample')
    data_h = pd.concat([data_h, meta_h], axis=1)
    
    data = pd.concat([data_c, data_h], axis=0)
    data = data.sort_index()
    
    #scale data
    scaler = StandardScaler()
    scaler.fit(data[features])
    data[features] = scaler.transform(data[features])
    data[features].mean()
    
    #add tumor fraction groups
    data['tfx_group'] = 'none'
    a = 0
    for b in tfx_groups:
        tfx_group_name = str(a)+'-'+str(b)+'TFx'
        data['tfx_group'] = np.where((data['phenotype']==1) & (data['fraction']>=a) & (data['fraction']<b),tfx_group_name,data['tfx_group'])
        a=b
    #if group maxes don't go all the way to 1, add a group > max val
    if b<1:
        tfx_group_name = '>'+str(b)+'TFx'
        data['tfx_group'] = np.where((data['phenotype']==1) & (data['phenotype']==1) & (data['fraction']>=b),tfx_group_name,data['tfx_group'])
    #specify tfx group for healthy donors
    data['tfx_group'] = np.where((data['phenotype']==0),'Healthy',data['tfx_group'])
    
    return(data,features)


In [4]:
def import_data_adjusted_allFeatures(in_file_c, meta_file_c, in_file_h, meta_file_h):
    import pandas as pd
    import numpy as np
    from sklearn.preprocessing import StandardScaler
    
    data_c = pd.read_csv(in_file_c, sep='\t', index_col=0)
    meta_c = pd.read_csv(meta_file_c, sep='\t', index_col='sample_name')

    #get features and exclude all other columns
    features = data_c.columns[(data_c.columns.str.startswith('central_cov')) | (data_c.columns.str.startswith('mean_cov')) | (data_c.columns.str.startswith('amplitude')) | (data_c.columns.str.startswith('nucleosome_spacing'))]
    print('Features',len(features))

    data_c = data_c.reset_index(drop=False)
    data_c[['sample','p','score']] = data_c['index'].str.split('_',2, expand=True)
    data_c = data_c.set_index('sample')
    data_c = pd.concat([data_c, meta_c], axis=1)
    
    data_h = pd.read_csv(in_file_h, sep='\t', index_col=0)
    meta_h = pd.read_csv(meta_file_h, sep='\t', index_col='sample_name')
    
    data_h = data_h.reset_index(drop=False)
    data_h[['sample','p','score']] = data_h['index'].str.split('_',2, expand=True)
    data_h = data_h.set_index('sample')
    data_h = pd.concat([data_h, meta_h], axis=1)
    
    data = pd.concat([data_c, data_h], axis=0)
    data = data.sort_index()
    
    #scale data
    scaler = StandardScaler()
    scaler.fit(data[features])
    data[features] = scaler.transform(data[features])
    data[features].mean()
    
    #add tumor fraction groups
    data['tfx_group'] = 'none'
    a = 0
    for b in tfx_groups:
        tfx_group_name = str(a)+'-'+str(b)+'TFx'
        data['tfx_group'] = np.where((data['phenotype']==1) & (data['fraction']>=a) & (data['fraction']<b),tfx_group_name,data['tfx_group'])
        a=b
    #if group maxes don't go all the way to 1, add a group > max val
    if b<1:
        tfx_group_name = '>'+str(b)+'TFx'
        data['tfx_group'] = np.where((data['phenotype']==1) & (data['phenotype']==1) & (data['fraction']>=b),tfx_group_name,data['tfx_group'])
    #specify tfx group for healthy donors
    data['tfx_group'] = np.where((data['phenotype']==0),'Healthy',data['tfx_group'])
    
    return(data,features)


In [5]:
#bootstrapping
def run_bootstrap_with_PCA(data,iterations,features,report_interval,hyperparameters):
    import time
    import sys
    import pandas as pd
    import numpy as np
    from sklearn.model_selection import StratifiedKFold
    from sklearn.linear_model import LogisticRegression
    from sklearn.model_selection import GridSearchCV
    from matplotlib import pyplot as plt
    from sklearn.decomposition import PCA

    start_time = time.time()

    probabilities = pd.DataFrame(index=data.index)
    c_vals = []
    coefs = pd.DataFrame(index=features)
    num_pcs = []
    train_indexes = []
    
    # Loop for each iteration
    for i in range(iterations):
            
        #bootstrap a training set with replacement
        X_train = data.sample(len(data), replace = True, random_state = i+100)[features]
        y_train = data.sample(len(data), replace = True, random_state = i+100)['phenotype']
        
        #the test set is all samples that aren't seen in the training data
        X_test = data[~(data.index.isin(X_train.index))][features]
        y_test = data[~(data.index.isin(X_train.index))]['phenotype']
        
        #print(len(X_train),len(X_train.index.unique()),len(X_test))
        
        #perform PCA on the training set
        n_components = min(len(features), len(X_train))
        pca = PCA(n_components=n_components, svd_solver='randomized', random_state = 100)
        PCs = pca.fit_transform(X_train[features])
        principal_components = pd.DataFrame(data = PCs, columns = ['PC_'+str(m) for m in np.arange(n_components)], index = X_train.index)
        
        #find the principle components that make up 80% of the varience
        for j in range(len(pca.explained_variance_ratio_)):
            current_sum = pca.explained_variance_ratio_[:j].sum()
            if current_sum>=fraction_variance:
                break
        #print('number of components:',j)
        pca_features = ['PC_'+str(m) for m in np.arange(0,j)]
        
        #apply to the test data
        test_PCs = pca.transform(X_test[features])
        test_principal_components = pd.DataFrame(data = test_PCs , columns = ['PC_'+str(m) for m in np.arange(n_components)], index = X_test.index)
        
        X_train = principal_components[pca_features]
        X_test = test_principal_components[pca_features]
        
        #10 fold cross validation on the training set
        cv = StratifiedKFold(n_splits=10, shuffle=True, random_state = i+100) 

        model = LogisticRegression(class_weight='balanced', max_iter=500, solver = 'liblinear')
        search = GridSearchCV(estimator=model, param_grid=hyperparameters, cv=cv, n_jobs = 1)
        search.fit(X_train, y_train)
        best_C = search.best_params_['C']

        ##train a new model on the full training dataset (is this the same as refit...?)
        model = LogisticRegression(class_weight='balanced', max_iter=500, C=best_C, solver = 'liblinear')
        model.fit(X_train, y_train)

        #predict the test data
        pred = model.predict(X_test)
        prob = model.predict_proba(X_test)
        #print(prob)
        #print(pd.Series(prob[:,1], index = X_test.index)) # BUT WHY THE SECOND VALUE AND NOT 0????

        #save results
        probabilities[i] = pd.Series(prob[:,1], index = X_test.index)
        c_vals.append(best_C)
        coefs[i] = pd.Series(model.coef_[0], index = pca_features) # WHAT IS THAT?
        num_pcs.append(j)
     
        train_indexes.append(list(X_train.index))
        
        if i%report_interval==0:
            print('iteration:',i, ', time (sec):',np.round(time.time()-start_time,2),'num_pcs:',j)
        if i%20==0:
            #prevent dfs from becoming too fragmented
            probabilities = probabilities.copy()
            coefs = coefs.copy()   
            sys.stdout.flush()

    probabilities = probabilities.merge(data[['phenotype']], left_index=True, right_index=True)

    return(probabilities,c_vals,coefs,num_pcs,train_indexes)


In [6]:
def get_AUC(probabilities,data,iterations):
    #get AUC and accuracy for each bootstrap
    from sklearn.metrics import roc_curve,auc
    import pandas as pd
    import numpy as np

    AUCs = pd.DataFrame()

    probabilities = probabilities.merge(data[['fraction','Status','Stage','tfx_group']], left_index=True, right_index=True)
    
    for i in range(iterations):
        current_dict = {}
        current = probabilities[~(probabilities[i].isnull())][['phenotype','fraction','Status','Stage','tfx_group',i]].copy()

        #overall accuracy and AUC
        group = 'overall'
        fpr,tpr,_ = roc_curve(current['phenotype'],current[i])
        AUC = auc(fpr,tpr)
        current_dict[group] = AUC
        del(AUC,group,fpr,tpr)

        #separate out the healthy samples to be used in every AUC
        healthy_df = current[current['phenotype']==0]
        cancer_df = current[current['phenotype']==1]
        del(current)
        
        for group,df in cancer_df.groupby('Status'):
            if group == 'Duodenal_Cancer':
                continue

            df2 = df.append(healthy_df, ignore_index=True)
            fpr,tpr,_ = roc_curve(df2['phenotype'],df2[i])
            AUC = auc(fpr,tpr)
            current_dict[group] = AUC
            del(AUC,group,fpr,tpr)
            
        for group,df in cancer_df.groupby('Stage'):
            if group == '0' or group == 'X':
                continue
            df2 = df.append(healthy_df, ignore_index=True)
            fpr,tpr,_ = roc_curve(df2['phenotype'],df2[i])
            AUC = auc(fpr,tpr)
            current_dict[group] = AUC
            del(AUC,group,fpr,tpr)
            
        #for group,df in cancer_df.groupby('tfx_group'):
        #    df2 = df.append(healthy_df, ignore_index=True)
        #    fpr,tpr,_ = roc_curve(df2['phenotype'],df2[i])
        #    AUC = auc(fpr,tpr)
        #    current_dict[group] = AUC
        #    del(AUC,group,fpr,tpr)
            
        AUCs = AUCs.append(pd.Series(current_dict), ignore_index=True)
        
    CIs = pd.DataFrame([AUCs.median(), AUCs.quantile(.025), AUCs.quantile(.975)]).T
    CIs = CIs.rename(columns = {'Unnamed 0':'median'})    
    return(AUCs,CIs)

# Parameter

In [7]:
in_file_c = "/data/gpfs-1/groups/ag_kircher/cfDNA-analysis/lea/cfDNA_classification_analyses/features/DELFI_breast_cancer_corrected_MIDPOINT_FFT_features.csv"
in_file_h = "/data/gpfs-1/groups/ag_kircher/cfDNA-analysis/lea/cfDNA_classification_analyses/features/DELFI_healthy_corrected_MIDPOINT_FFT_features.csv"

meta_file_c = "/data/gpfs-1/groups/ag_kircher/cfDNA-analysis/lea/cfDNA_classification_analyses/features/DELFI_breast_cancer_metadata.tsv"
meta_file_h = "/data/gpfs-1/groups/ag_kircher/cfDNA-analysis/lea/cfDNA_classification_analyses/features/DELFI_healthy_metadata.tsv"

iterations = 1000
report_interval = 50
fraction_variance = .8
tfx_groups = [0.03,0.05]

# PCA bootstrapping with my data

In [8]:
data,features = import_data_adjusted(in_file_c, meta_file_c, in_file_h, meta_file_h)
data.head()

Features 1131


,index,phenotype,central_coverage_NFKB2,mean_coverage_NFKB2,amplitude190_NFKB2,nucleosome_spacing_fft_NFKB2,central_coverage_TP73,mean_coverage_TP73,amplitude190_TP73,nucleosome_spacing_fft_TP73,...,Stage,Age,Status,% GC,Length,Median,≥ 1X,≥ 5X,fraction,tfx_group
EGAF00002727130,EGAF00002727130_h_MIDPOINT,0.0,-0.348951,0.935517,-1.469707,240.0,-0.663069,-0.007034,-0.041302,213.0,...,NaN,57.0,healthy,41%,140 bp,2.0X,89.0%,1.0%,0.00000,Healthy
EGAF00002727137,EGAF00002727137_h_MIDPOINT,0.0,0.058311,0.265062,-0.092322,240.0,-0.148291,-0.099124,-0.659806,192.0,...,NaN,48.0,healthy,42%,136 bp,8.0X,93.0%,83.0%,0.00000,Healthy
EGAF00002727139,EGAF00002727139_h_MIDPOINT,0.0,0.340260,0.203273,-0.459206,213.0,0.349726,-1.025165,-0.534208,240.0,...,NaN,75.0,healthy,42%,137 bp,3.0X,91.0%,9.0%,0.07122,Healthy
EGAF00002727144,EGAF00002727144_h_MIDPOINT,0.0,0.123577,0.954880,-1.105436,240.0,0.417166,0.950171,-0.517198,213.0,...,NaN,65.0,healthy,41%,140 bp,2.0X,89.0%,2.0%,0.04834,Healthy
EGAF00002727149,EGAF00002727149_h_MIDPOINT,0.0,0.378646,-0.055270,-1.264784,213.0,0.684335,-0.310720,0.176463,192.0,...,NaN,40.0,healthy,43%,139 bp,11.0X,93.0%,87.0%,0.00000,Healthy


In [9]:
# calculating probabilities
print('running '+str(iterations)+' logreg bootstrap iterations')
hyperparameters = {'C': [0.0001, 0.001,0.01,0.1,1,10,100,1000]}

probabilities,c_vals,coefs,num_pcs,train_indexes = run_bootstrap_with_PCA(data,iterations,features,report_interval,hyperparameters)    

running 1000 logreg bootstrap iterations
iteration: 0 , time (sec): 0.97 num_pcs: 35
iteration: 50 , time (sec): 164.5 num_pcs: 37
iteration: 100 , time (sec): 363.03 num_pcs: 31
iteration: 150 , time (sec): 585.12 num_pcs: 31
iteration: 200 , time (sec): 869.59 num_pcs: 33
iteration: 250 , time (sec): 1103.71 num_pcs: 33
iteration: 300 , time (sec): 1276.99 num_pcs: 35
iteration: 350 , time (sec): 1459.27 num_pcs: 34
iteration: 400 , time (sec): 1596.67 num_pcs: 35
iteration: 450 , time (sec): 1755.92 num_pcs: 35
iteration: 500 , time (sec): 1916.73 num_pcs: 36
iteration: 550 , time (sec): 2078.2 num_pcs: 37
iteration: 600 , time (sec): 2230.86 num_pcs: 35
iteration: 650 , time (sec): 2405.39 num_pcs: 31
iteration: 700 , time (sec): 2532.01 num_pcs: 31
iteration: 800 , time (sec): 2831.29 num_pcs: 34
iteration: 850 , time (sec): 2953.92 num_pcs: 33
iteration: 900 , time (sec): 3050.59 num_pcs: 40
iteration: 950 , time (sec): 3192.85 num_pcs: 37


In [10]:
# calculate AUC with confidence intervals
AUCs,CIs = get_AUC(probabilities,data,iterations)
CIs

,median,0.025,0.975
I,0.738095,0.128397,1.000000
II,0.822521,0.663442,0.940012
III,0.790476,0.466389,0.983447
breast_cancer,0.802859,0.653588,0.912303
overall,0.802859,0.653588,0.912303


## including all 4 features feature types (including nucleosome spacing)

In [11]:
data_all,features_all = import_data_adjusted_allFeatures(in_file_c, meta_file_c, in_file_h, meta_file_h)
data_all.head()

Features 1508


,index,phenotype,central_coverage_NFKB2,mean_coverage_NFKB2,amplitude190_NFKB2,nucleosome_spacing_fft_NFKB2,central_coverage_TP73,mean_coverage_TP73,amplitude190_TP73,nucleosome_spacing_fft_TP73,...,Stage,Age,Status,% GC,Length,Median,≥ 1X,≥ 5X,fraction,tfx_group
EGAF00002727130,EGAF00002727130_h_MIDPOINT,0.0,-0.348951,0.935517,-1.469707,1.398044,-0.663069,-0.007034,-0.041302,0.377780,...,NaN,57.0,healthy,41%,140 bp,2.0X,89.0%,1.0%,0.00000,Healthy
EGAF00002727137,EGAF00002727137_h_MIDPOINT,0.0,0.058311,0.265062,-0.092322,1.398044,-0.148291,-0.099124,-0.659806,-0.406840,...,NaN,48.0,healthy,42%,136 bp,8.0X,93.0%,83.0%,0.00000,Healthy
EGAF00002727139,EGAF00002727139_h_MIDPOINT,0.0,0.340260,0.203273,-0.459206,0.347620,0.349726,-1.025165,-0.534208,1.386579,...,NaN,75.0,healthy,42%,137 bp,3.0X,91.0%,9.0%,0.07122,Healthy
EGAF00002727144,EGAF00002727144_h_MIDPOINT,0.0,0.123577,0.954880,-1.105436,1.398044,0.417166,0.950171,-0.517198,0.377780,...,NaN,65.0,healthy,41%,140 bp,2.0X,89.0%,2.0%,0.04834,Healthy
EGAF00002727149,EGAF00002727149_h_MIDPOINT,0.0,0.378646,-0.055270,-1.264784,0.347620,0.684335,-0.310720,0.176463,-0.406840,...,NaN,40.0,healthy,43%,139 bp,11.0X,93.0%,87.0%,0.00000,Healthy


In [12]:
# calculating probabilities
print('running '+str(iterations)+' logreg bootstrap iterations')
hyperparameters = {'C': [0.0001, 0.001,0.01,0.1,1,10,100,1000]}

probabilities_all,c_vals_all,coefs_all,num_pcs_all,train_indexes_all = run_bootstrap_with_PCA(data_all,iterations,features_all,report_interval,hyperparameters)    

running 1000 logreg bootstrap iterations
iteration: 0 , time (sec): 5.95 num_pcs: 36
iteration: 50 , time (sec): 152.17 num_pcs: 38
iteration: 100 , time (sec): 325.07 num_pcs: 33
iteration: 150 , time (sec): 458.85 num_pcs: 33
iteration: 200 , time (sec): 662.44 num_pcs: 35
iteration: 250 , time (sec): 810.11 num_pcs: 35
iteration: 300 , time (sec): 933.87 num_pcs: 38
iteration: 350 , time (sec): 1086.65 num_pcs: 36
iteration: 400 , time (sec): 1254.15 num_pcs: 38
iteration: 450 , time (sec): 1388.3 num_pcs: 36
iteration: 500 , time (sec): 1549.91 num_pcs: 37
iteration: 550 , time (sec): 1690.8 num_pcs: 39
iteration: 600 , time (sec): 1819.85 num_pcs: 37
iteration: 650 , time (sec): 1969.85 num_pcs: 32
iteration: 700 , time (sec): 2074.41 num_pcs: 33
iteration: 750 , time (sec): 2200.55 num_pcs: 35
iteration: 800 , time (sec): 2338.1 num_pcs: 36
iteration: 850 , time (sec): 2431.04 num_pcs: 35
iteration: 900 , time (sec): 2551.88 num_pcs: 42
iteration: 950 , time (sec): 2705.58 num_pc

In [13]:
# calculate AUC with confidence intervals
AUCs_all,CIs_all = get_AUC(probabilities_all,data_all,iterations)
CIs_all

,median,0.025,0.975
I,0.733854,0.066702,1.000000
II,0.805263,0.642857,0.934070
III,0.763158,0.410079,0.986166
breast_cancer,0.783201,0.632222,0.900766
overall,0.783201,0.632222,0.900766
